In [1]:
import os
import time
import copy
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from nets.yolo4 import YoloBody
from nets.yolo_training import YOLOLoss, Generator

In [2]:
#---------------------------------------------------#
#   获得类和先验框
#---------------------------------------------------#
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    #print("class_names=",class_names)
    return class_names


def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    #print("anchors=",anchors)
    #print("np.array(anchors).reshape([-1,3,2])[::-1,:,:]=")
    #print(np.array(anchors).reshape([-1,3,2])[::-1,:,:])
    return np.array(anchors).reshape([-1,3,2])[::-1,:,:]


#---------------------------------------------------#
#   训练一个epoch
#---------------------------------------------------#
def fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen,genval, Epoch, cuda, optimizer, lr_scheduler):
    total_loss = 0
    val_loss = 0
    print('\n' + '-' * 10 + 'Train one epoch.' + '-' * 10)
    print('Epoch:'+ str(epoch+1) + '/' + str(Epoch))
    print('Start Training.')
    net.train()
    for iteration in range(epoch_size):
        start_time = time.time()
        images, targets = next(gen)
        with torch.no_grad():
            if cuda:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor)).cuda()
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
            else:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor))
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
        optimizer.zero_grad()
        outputs = net(images)
        losses = []
        for i in range(3):
            loss_item = yolo_losses[i](outputs[i], targets)
            losses.append(loss_item[0])
        loss = sum(losses)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss
        waste_time = time.time() - start_time
        if iteration == 0 or (iteration+1) % 10 == 0:
            print('step:' + str(iteration+1) + '/' + str(epoch_size) + ' || Total Loss: %.4f || %.4fs/step' % (total_loss/(iteration+1), waste_time))
    print('Finish Training.')
    '''        
    print('Start Validation.')
    net.eval()
    for iteration in range(epoch_size_val):
        images_val, targets_val = next(genval)

        with torch.no_grad():
            if cuda:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor)).cuda()
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            else:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor))
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            optimizer.zero_grad()
            outputs = net(images_val)
            losses = []
            for i in range(3):
                loss_item = yolo_losses[i](outputs[i], targets_val)
                losses.append(loss_item[0])
            loss = sum(losses)
            val_loss += loss
    print('Finish Validation')
    '''
    print('Total Loss: %.4f || Val Loss: %.4f ' % (total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)))
    
    return total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)

In [3]:
#-------------------------------#
#   输入的shape大小
#   显存比较小可以使用416x416
#   显存比较大可以使用608x608
#-------------------------------#
#input_shape = (416,416)
input_shape = (608, 608)

#-------------------------------#
#   tricks的使用设置
#-------------------------------#
Cosine_lr = True
mosaic = False
# 用于设定是否使用cuda
Cuda = True
smoooth_label = 0.03

#-------------------------------#
#   获得训练集和验证集的annotations
#   
#-------------------------------#
train_annotation_path = 'model_data/mask_train.txt'
val_annotation_path = 'model_data/mask_val.txt'

#-------------------------------#
#   获得先验框和类
#-------------------------------#
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/mask_classes.txt'   
class_names = get_classes(classes_path)
anchors = get_anchors(anchors_path)
num_classes = len(class_names)

In [4]:
# 创建模型
model = YoloBody(len(anchors[0]), num_classes)
print("len(anchors[0]=",len(anchors[0]),anchors[0])
# model_path = "model_data/yolov4_coco_pretrained_weights.pth"
model_path = "model_data/yolov4_maskdetect_weights0.pth"
# 加快模型训练的效率
print('Loading pretrained model weights.')
model_dict = model.state_dict()
pretrained_dict = torch.load(model_path,map_location=torch.device('cpu'))
pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) ==  np.shape(v)}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
print('Finished!')

if Cuda:
    net = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    net = net.cuda()
else:
    net = torch.nn.DataParallel(model)

# 建立loss函数
yolo_losses = []
for i in range(3):
    yolo_losses.append(YOLOLoss(np.reshape(anchors, [-1,2]), num_classes, \
                                (input_shape[1], input_shape[0]), smoooth_label, Cuda))
# read train lines and val lines
with open(train_annotation_path) as f:
    train_lines = f.readlines()
with open(val_annotation_path) as f:
    val_lines = f.readlines()
num_train = len(train_lines)
num_val = len(val_lines)

len(anchors[0]= 3 [[142. 110.]
 [192. 243.]
 [459. 401.]]
Loading pretrained model weights.
Finished!


In [ ]:
#------------------------------------#
#   先冻结backbone训练
#------------------------------------#
lr = 1e-3
Batch_size = 4
Init_Epoch = 0
Freeze_Epoch = 25
        
optimizer = optim.Adam(net.parameters(), lr, weight_decay=5e-4)
if Cosine_lr:
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
else:
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

gen = Generator(Batch_size, train_lines, (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
gen_val = Generator(Batch_size, val_lines, (input_shape[0], input_shape[1])).generate(mosaic = False)
                        
epoch_size = int(max(1, num_train//Batch_size//2.5)) if mosaic else max(1, num_train//Batch_size)
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = False

best_loss = 99999999.0
best_model_weights = copy.deepcopy(net.state_dict())
for epoch in range(Init_Epoch, Freeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Freeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item()) + '\n')
torch.save(best_model_weights, 'model_data/yolov4_maskdetect_weights0.pth')


----------Train one epoch.----------
Epoch:1/25
Start Training.


G:\aiplus\mask\MaskDetect-YOLOv4-PyTorch-master\nets\yolo_training.py:504: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tmp_targets = np.array(targets)


step:1/300 || Total Loss: 9.4138 || 5.0637s/step


c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


step:10/300 || Total Loss: 7.3929 || 0.9216s/step


In [5]:
#------------------------------------#
#   解冻backbone后训练
#------------------------------------#
lr = 1e-4
Batch_size = 1
Freeze_Epoch = 25
Unfreeze_Epoch = 50

optimizer = optim.Adam(net.parameters(), lr, weight_decay=5e-4)
if Cosine_lr:
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)
else:
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

gen = Generator(Batch_size, train_lines, (input_shape[0], input_shape[1])).generate(mosaic = mosaic)
gen_val = Generator(Batch_size, val_lines, (input_shape[0], input_shape[1])).generate(mosaic = False)
                        
epoch_size = int(max(1, num_train//Batch_size//2.5)) if mosaic else max(1, num_train//Batch_size)
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = True

best_loss = 99999999.0
best_model_weights = copy.deepcopy(net.state_dict())
for epoch in range(Freeze_Epoch, Unfreeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Unfreeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item() + '\n')
torch.save(best_model_weights, 'model_data/yolov4_maskdetect_weights1.pth')


----------Train one epoch.----------
Epoch:26/50
Start Training.
step:1/1200 || Total Loss: 6.7918 || 2.9059s/step
step:10/1200 || Total Loss: 15.9783 || 0.3181s/step
step:20/1200 || Total Loss: 12.1908 || 0.2962s/step
step:30/1200 || Total Loss: 9.4912 || 0.3002s/step
step:40/1200 || Total Loss: 11.6364 || 0.3002s/step
step:50/1200 || Total Loss: 12.7428 || 0.3560s/step
step:60/1200 || Total Loss: 11.9156 || 0.3271s/step
step:70/1200 || Total Loss: 10.9582 || 0.3072s/step
step:80/1200 || Total Loss: 9.9965 || 0.2992s/step
step:90/1200 || Total Loss: 10.8499 || 0.3032s/step
step:100/1200 || Total Loss: 12.5778 || 0.3311s/step
step:110/1200 || Total Loss: 11.7636 || 0.3032s/step
step:120/1200 || Total Loss: 11.1358 || 0.3361s/step
step:130/1200 || Total Loss: 10.8167 || 0.3122s/step
step:140/1200 || Total Loss: 17.0612 || 0.3301s/step
step:150/1200 || Total Loss: 16.5542 || 0.3052s/step
step:160/1200 || Total Loss: 16.4706 || 0.3391s/step
step:170/1200 || Total Loss: 15.8196 || 0.3102s

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


step:430/1200 || Total Loss: 11.8051 || 0.3062s/step
step:440/1200 || Total Loss: 11.6585 || 0.3111s/step
step:450/1200 || Total Loss: 11.8862 || 0.3062s/step
step:460/1200 || Total Loss: 11.8335 || 0.3012s/step
step:470/1200 || Total Loss: 11.6858 || 0.3062s/step
step:480/1200 || Total Loss: 11.7926 || 0.3072s/step
step:490/1200 || Total Loss: 11.7495 || 0.3221s/step
step:500/1200 || Total Loss: 11.8116 || 0.3401s/step
step:510/1200 || Total Loss: 11.6530 || 0.3132s/step
step:520/1200 || Total Loss: 11.5531 || 0.3181s/step
step:530/1200 || Total Loss: 11.4055 || 0.3172s/step
step:540/1200 || Total Loss: 11.3398 || 0.3201s/step
step:550/1200 || Total Loss: 11.4455 || 0.3142s/step
step:560/1200 || Total Loss: 11.3087 || 0.3042s/step
step:570/1200 || Total Loss: 11.1938 || 0.3082s/step
step:580/1200 || Total Loss: 11.0220 || 0.3391s/step
step:590/1200 || Total Loss: 10.8742 || 0.3102s/step
step:600/1200 || Total Loss: 10.8162 || 0.3042s/step
step:610/1200 || Total Loss: 10.7482 || 0.3301

step:760/1200 || Total Loss: 10.2956 || 0.3321s/step
step:770/1200 || Total Loss: 10.2008 || 0.3072s/step
step:780/1200 || Total Loss: 10.1717 || 0.3620s/step
step:790/1200 || Total Loss: 10.2503 || 0.3122s/step
step:800/1200 || Total Loss: 10.2181 || 0.3371s/step
step:810/1200 || Total Loss: 10.2687 || 0.3162s/step
step:820/1200 || Total Loss: 10.2208 || 0.3082s/step
step:830/1200 || Total Loss: 10.1395 || 0.3331s/step
step:840/1200 || Total Loss: 10.0522 || 0.3391s/step
step:850/1200 || Total Loss: 9.9786 || 0.3132s/step
step:860/1200 || Total Loss: 9.9368 || 0.4668s/step
step:870/1200 || Total Loss: 9.9179 || 0.3162s/step
step:880/1200 || Total Loss: 9.9031 || 0.3331s/step
step:890/1200 || Total Loss: 9.8743 || 0.3331s/step
step:900/1200 || Total Loss: 9.8270 || 0.3381s/step
step:910/1200 || Total Loss: 9.7675 || 0.3132s/step
step:920/1200 || Total Loss: 9.7401 || 0.3142s/step
step:930/1200 || Total Loss: 9.6575 || 0.3451s/step
step:940/1200 || Total Loss: 9.6485 || 0.3301s/step
ste

step:1100/1200 || Total Loss: 8.6916 || 0.3132s/step
step:1110/1200 || Total Loss: 8.7701 || 0.3511s/step
step:1120/1200 || Total Loss: 8.7455 || 0.3979s/step
step:1130/1200 || Total Loss: 8.6888 || 0.3092s/step
step:1140/1200 || Total Loss: 8.6588 || 0.3092s/step
step:1150/1200 || Total Loss: 8.6444 || 0.4597s/step
step:1160/1200 || Total Loss: 8.5931 || 0.3092s/step
step:1170/1200 || Total Loss: 8.5645 || 0.3441s/step
step:1180/1200 || Total Loss: 8.5287 || 0.3141s/step
step:1190/1200 || Total Loss: 8.4927 || 0.3750s/step
step:1200/1200 || Total Loss: 8.4383 || 0.3152s/step
Finish Training.
Total Loss: 8.4312 || Val Loss: 0.0000 

----------Train one epoch.----------
Epoch:29/50
Start Training.
step:1/1200 || Total Loss: 9.0033 || 0.3491s/step
step:10/1200 || Total Loss: 9.5078 || 0.3102s/step
step:20/1200 || Total Loss: 6.8830 || 0.3102s/step
step:30/1200 || Total Loss: 10.1868 || 0.3042s/step
step:40/1200 || Total Loss: 8.7219 || 0.3481s/step
step:50/1200 || Total Loss: 7.4202 || 0

...

step:610/1200 || Total Loss: 6.0112 || 0.4827s/step
step:620/1200 || Total Loss: 5.9795 || 0.5375s/step
step:630/1200 || Total Loss: 5.9203 || 0.6572s/step
step:640/1200 || Total Loss: 5.9377 || 0.5216s/step
step:650/1200 || Total Loss: 6.0686 || 0.5286s/step
step:660/1200 || Total Loss: 6.0314 || 0.5406s/step
step:670/1200 || Total Loss: 5.9979 || 0.5166s/step
step:680/1200 || Total Loss: 6.1066 || 0.5366s/step
step:690/1200 || Total Loss: 6.0484 || 0.5705s/step
step:700/1200 || Total Loss: 6.0818 || 0.4907s/step
step:710/1200 || Total Loss: 6.0617 || 0.5685s/step
step:720/1200 || Total Loss: 6.0096 || 0.5795s/step
step:730/1200 || Total Loss: 6.0029 || 0.5056s/step
step:740/1200 || Total Loss: 6.1106 || 0.7091s/step
step:750/1200 || Total Loss: 6.1673 || 0.5635s/step
step:760/1200 || Total Loss: 6.1221 || 0.5136s/step
step:770/1200 || Total Loss: 6.0830 || 0.5894s/step
step:780/1200 || Total Loss: 6.0564 || 0.5455s/step
step:790/1200 || Total Loss: 6.0741 || 0.5406s/step
step:800/120

step:950/1200 || Total Loss: 5.8879 || 0.7874s/step
step:960/1200 || Total Loss: 5.8490 || 0.6423s/step
step:970/1200 || Total Loss: 5.8783 || 0.6951s/step
step:980/1200 || Total Loss: 6.1644 || 0.6343s/step
step:990/1200 || Total Loss: 6.1546 || 0.7663s/step
step:1000/1200 || Total Loss: 6.2327 || 0.7279s/step
step:1010/1200 || Total Loss: 6.2048 || 0.7588s/step
step:1020/1200 || Total Loss: 6.3053 || 0.6897s/step
step:1030/1200 || Total Loss: 6.2839 || 0.7522s/step
step:1040/1200 || Total Loss: 6.2346 || 0.7094s/step
step:1050/1200 || Total Loss: 6.1941 || 0.6941s/step
step:1060/1200 || Total Loss: 6.2577 || 0.7450s/step
step:1070/1200 || Total Loss: 6.2169 || 0.6914s/step
step:1080/1200 || Total Loss: 6.1773 || 0.6667s/step
step:1090/1200 || Total Loss: 6.1394 || 0.6782s/step
step:1100/1200 || Total Loss: 6.1250 || 0.7482s/step
step:1110/1200 || Total Loss: 6.1067 || 0.7106s/step
step:1120/1200 || Total Loss: 6.1383 || 0.6852s/step
step:1130/1200 || Total Loss: 6.1235 || 0.6271s/ste

step:60/1200 || Total Loss: 4.2139 || 0.3760s/step
step:70/1200 || Total Loss: 4.0969 || 0.3864s/step
step:80/1200 || Total Loss: 4.4577 || 0.3810s/step
step:90/1200 || Total Loss: 4.3615 || 0.3765s/step
step:100/1200 || Total Loss: 4.2131 || 0.3466s/step
step:110/1200 || Total Loss: 4.2140 || 0.3461s/step
step:120/1200 || Total Loss: 4.1797 || 0.3930s/step
step:130/1200 || Total Loss: 4.1545 || 0.3600s/step
step:140/1200 || Total Loss: 4.0303 || 0.3551s/step
step:150/1200 || Total Loss: 5.1237 || 0.3635s/step
step:160/1200 || Total Loss: 4.9440 || 0.3678s/step
step:170/1200 || Total Loss: 4.8219 || 0.3342s/step
step:180/1200 || Total Loss: 5.1457 || 0.3850s/step
step:190/1200 || Total Loss: 5.0693 || 0.3720s/step
step:200/1200 || Total Loss: 5.1528 || 0.3491s/step
step:210/1200 || Total Loss: 5.1440 || 0.3630s/step
step:220/1200 || Total Loss: 5.3790 || 0.4059s/step
step:230/1200 || Total Loss: 5.7018 || 0.3580s/step
step:240/1200 || Total Loss: 5.6863 || 0.3760s/step
step:250/1200 ||

step:410/1200 || Total Loss: 4.4742 || 0.6269s/step
step:420/1200 || Total Loss: 4.5095 || 0.6219s/step
step:430/1200 || Total Loss: 4.8157 || 0.6103s/step
step:440/1200 || Total Loss: 4.7729 || 0.5715s/step
step:450/1200 || Total Loss: 4.7786 || 0.6308s/step
step:460/1200 || Total Loss: 4.7720 || 0.6522s/step
step:470/1200 || Total Loss: 4.7354 || 0.5845s/step
step:480/1200 || Total Loss: 4.7135 || 0.6241s/step
step:490/1200 || Total Loss: 4.9372 || 0.6212s/step
step:500/1200 || Total Loss: 5.1505 || 0.7141s/step
step:510/1200 || Total Loss: 5.1476 || 0.6014s/step
step:520/1200 || Total Loss: 5.0751 || 0.6079s/step
step:530/1200 || Total Loss: 5.0380 || 0.6622s/step
step:540/1200 || Total Loss: 5.0094 || 0.7739s/step
step:550/1200 || Total Loss: 5.0282 || 0.6134s/step
step:560/1200 || Total Loss: 4.9729 || 0.5788s/step
step:570/1200 || Total Loss: 5.0520 || 0.6592s/step
step:580/1200 || Total Loss: 5.0467 || 0.6842s/step
step:590/1200 || Total Loss: 5.0605 || 0.6762s/step
step:600/120

step:750/1200 || Total Loss: 5.5739 || 0.4793s/step
step:760/1200 || Total Loss: 5.6041 || 0.4289s/step
step:770/1200 || Total Loss: 5.5800 || 0.3590s/step
step:780/1200 || Total Loss: 5.6685 || 0.4488s/step
step:790/1200 || Total Loss: 5.6549 || 0.4199s/step
step:800/1200 || Total Loss: 5.6209 || 0.4135s/step
step:810/1200 || Total Loss: 5.5756 || 0.4950s/step
step:820/1200 || Total Loss: 5.5452 || 0.4441s/step
step:830/1200 || Total Loss: 5.5051 || 0.4702s/step
step:840/1200 || Total Loss: 5.4976 || 0.5066s/step
step:850/1200 || Total Loss: 5.7163 || 0.4318s/step
step:860/1200 || Total Loss: 5.6697 || 0.4129s/step
step:870/1200 || Total Loss: 5.6273 || 0.4543s/step
step:880/1200 || Total Loss: 5.6406 || 0.4249s/step
step:890/1200 || Total Loss: 5.5895 || 0.4304s/step
step:900/1200 || Total Loss: 5.5447 || 0.4209s/step
step:910/1200 || Total Loss: 5.5310 || 0.3835s/step
step:920/1200 || Total Loss: 5.4837 || 0.4688s/step
step:930/1200 || Total Loss: 5.4443 || 0.4239s/step
step:940/120